In [ ]:
from urllib import FancyURLopener, quote_plus
from lxml.html import parse, document_fromstring
from lxml.cssselect import CSSSelector
import pandas as pd
import re

#link = 'http://www.cian.ru/cat.php?deal_type=sale&engine_version=2&foot_min=25&metro%5B0%5D=245&minfloor=2&offer_type=flat&only_foot=2&room2=1'
link = 'http://www.cian.ru/kupit-kvartiru-moskva-brateevo-0486/'
urlOpener = FancyURLopener({})
html = urlOpener.open(link).read()
document = document_fromstring(html)
pages = ['http://www.cian.ru'+ x.attrib['href'] for x in document.cssselect('.pager_pages a')]
i = 1
cnt_change = 1
while cnt_change !=0 or i == 1000:
    len_old = len(pages)
    link_tmp = pages[-1]
    html_tmp = urlOpener.open(link_tmp).read()
    document_tmp = document_fromstring(html_tmp)
    pages_tmp = ['http://www.cian.ru'+ x.attrib['href'] for x in document_tmp.cssselect('.pager_pages a') if x.attrib['href']!=link]
    pages = set(pages)
    for x in pages_tmp:
        pages.add(x)
    pages = list(pages)
    len_new = len(pages)
    cnt_change = len_new - len_old
    i=i+1
pages.append(link)

urls = []
#ids = []
for link_tmp in pages:
    html_tmp = urlOpener.open(link_tmp).read()
    document_tmp = document_fromstring(html_tmp)
    flats_info = document_tmp.cssselect('#content')
    flat = flats_info[0].cssselect('.serp-list .serp-item')
    for i in range(len(flat)):
        #ids.append(flat[i].attrib['oid'])
        urls.append(flat[i].attrib['href'])
urls = set(urls)
df = pd.DataFrame(columns=('url','cnt_rooms', 'price_all', 'total_area','address','metro','dt_metro', 'popularity','time_of_posting',
                           'floor', 'living_area','kitchen_area','elevator','toilet','message','year_of_construction', 'house_type',
                           'cnt_floors','cnt_entrance', 'cnt_flats','avg_price_m2_in_house','avg_rental_rate','name_of_district',
                           'district_population','avg_age','avg_price_m2_in_district'))
for i,u in enumerate(urls):   
    #dict_tmp = {'url':None,'price_all':None,'square_all':None,'address':None}
    res = []
    flat_tmp = document_fromstring(urlOpener.open(u).read())
    price_all = flat_tmp.cssselect('#price_rur')
    square_all = flat_tmp.cssselect('.object_descr_props tr:nth-child(5) td')
    address = flat_tmp.cssselect('.object_descr_addr a')
    #dict_tmp['url'] = urls[1]
    res.append(u)
    res.append(flat_tmp.cssselect('.object_descr_title')[0].text_content().replace(' ','').replace('\n',''))
    #dict_tmp['price_all'] = float(price_all[0].text_content().replace(',','.'))
    res.append(float(price_all[0].text_content().replace(',','.')))
    s = square_all[0]
    s = s.text_content()
    #dict_tmp['square_all'] = int(''.join(x for x in s.strip() if x.isdigit())[0:2])
    res.append(int(''.join(x for x in s.strip() if x.isdigit())[0:2]))
    adrr = [f.text_content() for f in address]
    #dict_tmp['address'] = ' '.join(x for x in adrr)
    res.append(' '.join(x for x in adrr))
    res.append(flat_tmp.cssselect('.object_descr_metro a')[0].text_content())
    if len(flat_tmp.cssselect('.object_item_metro_comment')) != 0:
           res.append(re.sub(' +',' ',flat_tmp.cssselect('.object_item_metro_comment')[0].text_content().replace('\n','')))
    else:
        res.append('')
    res.append(flat_tmp.cssselect('.object_descr_dt_added a')[0].text_content())
    res.append(flat_tmp.cssselect('.object_descr_dt_added')[1].text_content())
    res.append(flat_tmp.cssselect('.object_descr_props tr:nth-child(2) td')[0].text_content().replace(' ','').replace('\n',''))
    res.append(flat_tmp.cssselect('.object_descr_props tr:nth-child(7) td')[0].text_content().replace(' ','').replace('\n',''))
    res.append(flat_tmp.cssselect('.object_descr_props tr:nth-child(8) td')[0].text_content().replace(' ','').replace('\n',''))
    res.append(flat_tmp.cssselect('.object_descr_props tr:nth-child(11) td')[0].text_content().replace(' ','').replace('\n',''))
    res.append(flat_tmp.cssselect('.object_descr_props tr:nth-child(9) td')[0].text_content().replace(' ','').replace('\n',''))
    s = re.sub(' +',' ',flat_tmp.cssselect('.object_descr_text')[0].text_content().replace('\n',''))
    res.append(s[0:(s.find('/*')-1)])
    if len(flat_tmp.cssselect('.bti__data tbody')) != 0:        
        res.append(flat_tmp.cssselect('.bti__data tbody')[0].cssselect('tr:nth-child(1) td')[0].text_content())
        res.append(flat_tmp.cssselect('.bti__data tbody')[0].cssselect('tr:nth-child(2) td')[0].text_content())
        res.append(flat_tmp.cssselect('.bti__data tbody')[0].cssselect('tr:nth-child(3) td')[0].text_content())
        res.append(flat_tmp.cssselect('.bti__data tbody')[0].cssselect('tr:nth-child(4) td')[0].text_content())
        res.append(flat_tmp.cssselect('.bti__data tbody')[0].cssselect('tr:nth-child(5) td')[0].text_content())
        res.append(flat_tmp.cssselect('.bti__data tbody')[0].cssselect('tr:nth-child(6) td')[0].text_content())
        res.append(flat_tmp.cssselect('.bti__data tbody')[0].cssselect('tr:nth-child(8) td')[0].text_content())
        res.append(flat_tmp.cssselect('.bti__data tbody')[1].cssselect('tr:nth-child(1) td')[0].text_content())
        res.append(flat_tmp.cssselect('.bti__data tbody')[1].cssselect('tr:nth-child(3) td')[0].text_content())
        res.append(flat_tmp.cssselect('.bti__data tbody')[1].cssselect('tr:nth-child(4) td')[0].text_content())
        res.append(re.sub(' +',' ',flat_tmp.cssselect('.bti__data tbody')[1].cssselect('tr:nth-child(5) td')[0].text_content().replace('\n','')))
    else:
        for j in range(11):
            res.append('')            
    df.loc[i] = res

In [54]:
#импорт необходимых библиотек 
from urllib import FancyURLopener, quote_plus
from lxml.html import parse, document_fromstring
from lxml.cssselect import CSSSelector
import pandas as pd
import re

def best_flats(link):
    urlOpener = FancyURLopener({})
    main_page_doc = document_fromstring(urlOpener.open(link).read())
    pages = ['http://www.cian.ru'+ x.attrib['href'] for x in main_page_doc.cssselect('.pager_pages a')]

    cnt_change = 1
    while cnt_change !=0:
        len_old = len(pages)
        page_last = pages[-1]
        page_last_doc = document_fromstring(urlOpener.open(page_last).read())
        pages_delta = ['http://www.cian.ru'+ x.attrib['href'] 
                       for x in page_last_doc.cssselect('.pager_pages a')
                       if x.attrib['href']!=link]
        pages = set(pages)
        for x in pages_delta:
            pages.add(x)
        pages = list(pages)
        len_new = len(pages)
        cnt_change = len_new - len_old
    pages.append(link)
    
    urls = []
    for p in pages:
        flat_doc = document_fromstring(urlOpener.open(p).read())
        flats_info = flat_doc.cssselect('#content')
        flat = flats_info[0].cssselect('.serp-list .serp-item')
        for i in range(len(flat)):
            urls.append(flat[i].attrib['href'])
    urls = set(urls)
    df = pd.DataFrame(columns=('url','cnt_rooms', 'price_all', 'total_area','address','metro','dt_metro',
                               'popularity','time_of_posting', 'floor', 'living_area','kitchen_area',
                               'elevator','toilet','message','year_of_construction', 'house_type',
                               'cnt_floors','cnt_entrance', 'cnt_flats','avg_price_m2_in_house','avg_rental_rate',
                               'name_of_district','district_population','avg_age','avg_price_m2_in_district'))
    for i,u in enumerate(urls):   
        res = []
        flat_tmp = document_fromstring(urlOpener.open(u).read())
        price_all = flat_tmp.cssselect('#price_rur')
        square_all = flat_tmp.cssselect('.object_descr_props tr:nth-child(5) td')
        address = flat_tmp.cssselect('.object_descr_addr a')
        res.append(u)
        if len(flat_tmp.cssselect('.object_descr_title')) != 0:
            res.append(flat_tmp.cssselect('.object_descr_title')[0].text_content().replace(' ','').replace('\n',''))
        else:
            res.append('')
        if len(price_all) != 0:
            res.append(float(price_all[0].text_content().replace(',','.')))
        else:
            res.append('')
        if len(square_all) != 0:
            s = square_all[0]
            s = s.text_content()
            res.append(''.join(x for x in s.strip() if x.isdigit())[0:2])
        else:
            res.append('')
        if len(address) != 0:
            adrr = [f.text_content() for f in address]
            res.append(' '.join(x for x in adrr))
        else:
            res.append('')
        if len(flat_tmp.cssselect('.object_descr_metro a')) != 0:
            res.append(flat_tmp.cssselect('.object_descr_metro a')[0].text_content())
        else:
            res.append('')
        if len(flat_tmp.cssselect('.object_item_metro_comment')) != 0:
               res.append(re.sub(' +',' ',flat_tmp.cssselect('.object_item_metro_comment')[0].text_content().replace('\n','')))
        else:
            res.append('')
        if len(flat_tmp.cssselect('.object_descr_dt_added a')) != 0:
            res.append(flat_tmp.cssselect('.object_descr_dt_added a')[0].text_content())
        else:
            res.append('')
        if len(flat_tmp.cssselect('.object_descr_dt_added')) > 1:
            res.append(flat_tmp.cssselect('.object_descr_dt_added')[1].text_content())
        else:
            res.append('')        
        res.append(flat_tmp.cssselect('.object_descr_props tr:nth-child(2) td')[0].text_content().replace(' ','').replace('\n',''))
        res.append(flat_tmp.cssselect('.object_descr_props tr:nth-child(7) td')[0].text_content().replace(' ','').replace('\n',''))
        res.append(flat_tmp.cssselect('.object_descr_props tr:nth-child(8) td')[0].text_content().replace(' ','').replace('\n',''))
        res.append(flat_tmp.cssselect('.object_descr_props tr:nth-child(11) td')[0].text_content().replace(' ','').replace('\n',''))
        res.append(flat_tmp.cssselect('.object_descr_props tr:nth-child(9) td')[0].text_content().replace(' ','').replace('\n',''))
        s = re.sub(' +',' ',flat_tmp.cssselect('.object_descr_text')[0].text_content().replace('\n',''))
        res.append(s[0:(s.find('/*')-1)])
        if len(flat_tmp.cssselect('.bti__data tbody')) != 0:        
            res.append(flat_tmp.cssselect('.bti__data tbody')[0].cssselect('tr:nth-child(1) td')[0].text_content())
            res.append(flat_tmp.cssselect('.bti__data tbody')[0].cssselect('tr:nth-child(2) td')[0].text_content())
            res.append(flat_tmp.cssselect('.bti__data tbody')[0].cssselect('tr:nth-child(3) td')[0].text_content())
            res.append(flat_tmp.cssselect('.bti__data tbody')[0].cssselect('tr:nth-child(4) td')[0].text_content())
            res.append(flat_tmp.cssselect('.bti__data tbody')[0].cssselect('tr:nth-child(5) td')[0].text_content())
            res.append(flat_tmp.cssselect('.bti__data tbody')[0].cssselect('tr:nth-child(6) td')[0].text_content())
            res.append(flat_tmp.cssselect('.bti__data tbody')[0].cssselect('tr:nth-child(8) td')[0].text_content())
            res.append(flat_tmp.cssselect('.bti__data tbody')[1].cssselect('tr:nth-child(1) td')[0].text_content())
            res.append(flat_tmp.cssselect('.bti__data tbody')[1].cssselect('tr:nth-child(3) td')[0].text_content())
            res.append(flat_tmp.cssselect('.bti__data tbody')[1].cssselect('tr:nth-child(4) td')[0].text_content())
            res.append(re.sub(' +',' ',flat_tmp.cssselect('.bti__data tbody')[1].cssselect('tr:nth-child(5) td')[0].text_content().replace('\n','')))
        else:
            for j in range(11):
                res.append('')            
        df.loc[i] = res
    return df

In [55]:
brateevo = best_flats('http://www.cian.ru/kupit-kvartiru-moskva-brateevo-0486/')
print 'brateevo'
zyablikovo = best_flats('http://www.cian.ru/kupit-kvartiru-moskva-zyablikovo-0489/')
print 'zyablikovo'

brateevo
zyablikovo


In [56]:
orehovo_borisovo_yuzhnoe = best_flats('http://www.cian.ru/kupit-kvartiru-moskva-orehovo-borisovo-yuzhnoe-0495/')

In [57]:
brateevo.to_excel(u'C:\\Users\\Valery\\Desktop\\cian\\brateevo_110916.xlsx')
zyablikovo.to_excel(u'C:\\Users\\Valery\\Desktop\\cian\\zyablikovo_110916.xlsx')
orehovo_borisovo_yuzhnoe.to_excel(u'C:\\Users\\Valery\\Desktop\\cian\\orehovo_borisovo_yuzhnoe_110916.xlsx')

In [58]:
del brateevo, zyablikovo, orehovo_borisovo_yuzhnoe

In [59]:
orehovo_borisovo_severnoe = best_flats('http://www.cian.ru/kupit-kvartiru-vtorichka-moskva-orehovo-borisovo-severnoe-0494/')
orehovo_borisovo_severnoe.to_excel(u'C:\\Users\\Valery\\Desktop\\cian\\orehovo_borisovo_severnoe_110916.xlsx')
del orehovo_borisovo_severnoe

In [60]:
moskvoreche_saburovo = best_flats('http://www.cian.ru/kupit-kvartiru-moskva-moskvoreche-saburovo-0490/')
moskvoreche_saburovo.to_excel(u'C:\\Users\\Valery\\Desktop\\cian\\moskvoreche_saburovo_060916.xlsx')
del moskvoreche_saburovo

In [61]:
caricyno = best_flats('http://www.cian.ru/kupit-kvartiru-moskva-caricyno-0496/')
caricyno.to_excel(u'C:\\Users\\Valery\\Desktop\\cian\\caricyno_110916.xlsx')
del caricyno

In [62]:
nagatino_sadovniki = best_flats('http://www.cian.ru/kupit-kvartiru-moskva-nagatino-sadovniki-0491/')
nagatino_sadovniki.to_excel(u'C:\\Users\\Valery\\Desktop\\cian\\nagatino_sadovniki_110916.xlsx')
del nagatino_sadovniki

In [42]:
link_search = 'http://www.cian.ru/cat.php?acontext=%D0%B0%D0%BF%D0%B0%D1%80%D1%82%D0%B0%D0%BC%D0%B5%D0%BD%D1%82%D1%8B&currency=2&deal_type=sale&engine_version=2&foot_min=20&maxprice=8000000&metro%5B0%5D=1&metro%5B10%5D=20&metro%5B11%5D=21&metro%5B12%5D=22&metro%5B13%5D=27&metro%5B14%5D=32&metro%5B15%5D=38&metro%5B16%5D=39&metro%5B17%5D=40&metro%5B18%5D=41&metro%5B19%5D=43&metro%5B1%5D=2&metro%5B20%5D=45&metro%5B21%5D=46&metro%5B22%5D=47&metro%5B23%5D=48&metro%5B24%5D=49&metro%5B25%5D=50&metro%5B26%5D=52&metro%5B27%5D=53&metro%5B28%5D=54&metro%5B29%5D=55&metro%5B2%5D=4&metro%5B30%5D=56&metro%5B31%5D=58&metro%5B32%5D=61&metro%5B33%5D=64&metro%5B34%5D=65&metro%5B35%5D=66&metro%5B36%5D=67&metro%5B37%5D=68&metro%5B38%5D=71&metro%5B39%5D=76&metro%5B3%5D=5&metro%5B40%5D=77&metro%5B41%5D=78&metro%5B42%5D=80&metro%5B43%5D=82&metro%5B44%5D=84&metro%5B45%5D=85&metro%5B46%5D=86&metro%5B47%5D=88&metro%5B48%5D=89&metro%5B49%5D=90&metro%5B4%5D=8&metro%5B50%5D=92&metro%5B51%5D=95&metro%5B52%5D=96&metro%5B53%5D=98&metro%5B54%5D=100&metro%5B55%5D=101&metro%5B56%5D=103&metro%5B57%5D=105&metro%5B58%5D=107&metro%5B59%5D=111&metro%5B5%5D=8&metro%5B60%5D=113&metro%5B61%5D=114&metro%5B62%5D=115&metro%5B63%5D=115&metro%5B64%5D=117&metro%5B65%5D=119&metro%5B66%5D=121&metro%5B67%5D=123&metro%5B68%5D=124&metro%5B69%5D=125&metro%5B6%5D=12&metro%5B70%5D=129&metro%5B71%5D=130&metro%5B72%5D=132&metro%5B73%5D=137&metro%5B74%5D=144&metro%5B75%5D=145&metro%5B76%5D=146&metro%5B77%5D=148&metro%5B78%5D=149&metro%5B79%5D=150&metro%5B7%5D=13&metro%5B80%5D=152&metro%5B81%5D=155&metro%5B82%5D=159&metro%5B83%5D=238&metro%5B84%5D=239&metro%5B85%5D=240&metro%5B86%5D=245&metro%5B87%5D=283&metro%5B8%5D=15&metro%5B9%5D=18&offer_type=flat&only_foot=2'

search = best_flats(link_search)
search.to_excel(u'C:\\Users\\Valery\\Desktop\\cian\\search_060916.xlsx')
del search

In [44]:
data = pd.read_excel(u'C:\\Users\\Valery\\Desktop\\cian\\caricyno_060916.xlsx')

In [51]:
data.cnt_rooms.apply(lambda x: int(x[0]))

UnicodeEncodeError: 'decimal' codec can't encode character u'\u0441' in position 0: invalid decimal Unicode string

с
